05 交互作用特徴量
==============

* `交互作用特徴量`：複数の特徴量の積として定義される

    * 複数の特徴量を組み合わせて新たな特徴量を作成する
    
    * その中でも2つの特徴量の積は、`ペアワイズ交互作用特徴量`と呼ばれる
    
    * 例)郵便番号と年齢層の2つの特徴量から交互作用特徴量を作成すると、その値は「郵便番号98121かつ年齢が18歳から35歳まで」のようになる
    
        * 2つの特徴量を組み合わせることで、個々の特徴量を使った場合よりもターゲット変数をうまく表現できる場合がある
        


* 2つの特徴量を組み合わせることで、個々の特徴量を使った場合よりもターゲット変数をうまく表現できる場合がある

* 線形モデルで交互作用特徴量を使う場合には、明示的にモデルに組み込む必要がある(決定木に基づくモデルでは、その必要はない)

* 線形モデルは、入力特徴量$x_1$、$x_2$、...、$x_n$の線形変換によって、ターゲット変数$y$を予測する

\begin{eqnarray}
y = w_1x_1 + w_2x_2 + ... + w_nx_n
\end{eqnarray}

* この線形モデルの式に交互作用特徴量を組み込むのは簡単

\begin{eqnarray}
y = w_1x_1 + w_2x_2 + ... + w_nx_n + w_{1, 1}x_1x_1 + w_{1, 2}x_1x_2 + w_{1, 3}x_1x_3 + ...
\end{eqnarray}

* これにより、モデルに交互作用(2つ以上の特徴量による複合的な影響)を組み込むことができる

    * 特徴量が数値の場合、$x_1x_2$は通常の掛け算の意味
    
    * $x_1$と$x_2$が二値特徴量の場合、$x_1x_2$は論理積($x_1 AND x_2$)となる
    


* 例)ユーザのプロフィール情報を使って商品を買うかどうかを予測するタスクを考える

    * この時、居住地域と年齢を別々に考えるより、「シアトルに住む35歳」という情報の方が役にたつ場合がある

* 例)ペアワイズ交互作用特徴量を使って、Online News Popularityデータセットからニュース記事のシェア数を予測する

    * 結果として、交互作用特徴量を加えたモデルは特徴量をそれぞれ単一に用いたモデルより精度が上がる
    
    * また、どちらのモデルも記事中の単語数だけを特徴量としたモデルよりも良い性能となった

In [4]:
import pandas as pd
from sklearn import linear_model
from sklearn.model_selection import train_test_split
import sklearn.preprocessing as preproc

df = pd.read_csv('/Users/kunii.sotaro/Downloads/OnlineNewsPopularity/OnlineNewsPopularity.csv', delimiter=', ', engine='python')
df.columns

Index(['url', 'timedelta', 'n_tokens_title', 'n_tokens_content',
       'n_unique_tokens', 'n_non_stop_words', 'n_non_stop_unique_tokens',
       'num_hrefs', 'num_self_hrefs', 'num_imgs', 'num_videos',
       'average_token_length', 'num_keywords', 'data_channel_is_lifestyle',
       'data_channel_is_entertainment', 'data_channel_is_bus',
       'data_channel_is_socmed', 'data_channel_is_tech',
       'data_channel_is_world', 'kw_min_min', 'kw_max_min', 'kw_avg_min',
       'kw_min_max', 'kw_max_max', 'kw_avg_max', 'kw_min_avg', 'kw_max_avg',
       'kw_avg_avg', 'self_reference_min_shares', 'self_reference_max_shares',
       'self_reference_avg_sharess', 'weekday_is_monday', 'weekday_is_tuesday',
       'weekday_is_wednesday', 'weekday_is_thursday', 'weekday_is_friday',
       'weekday_is_saturday', 'weekday_is_sunday', 'is_weekend', 'LDA_00',
       'LDA_01', 'LDA_02', 'LDA_03', 'LDA_04', 'global_subjectivity',
       'global_sentiment_polarity', 'global_rate_positive_words',
     

In [5]:
features = ['n_tokens_title', 'n_tokens_content', 'n_unique_tokens',
            'n_non_stop_words', 'n_non_stop_unique_tokens', 'num_hrefs',
            'num_self_hrefs', 'num_imgs', 'num_videos', 'average_token_length',
            'num_keywords', 'data_channel_is_lifestyle', 'data_channel_is_entertainment',
            'data_channel_is_bus', 'data_channel_is_socmed', 'data_channel_is_tech',
            'data_channel_is_world']

X = df[features]
y = df[['shares']]

# ペアワイズ交互作用特徴量を作成する（定数項（bias）は含めない）
X2 = preproc.PolynomialFeatures(include_bias=False).fit_transform(X)
X2.shape

(39644, 170)

In [6]:
# 両方の特徴量セットを訓練データとテストデータに分ける
X1_train, X1_test, X2_train, X2_test, y_train, y_test = \
    train_test_split(X, X2, y, test_size=0.3, random_state=123)

def evaluate_feature(X_train, X_test, y_train, y_test):
    """
    訓練データに対して線形回帰モデルを適合し
    テストデータに対してスコアを算出する
    """
    model = linear_model.LinearRegression().fit(X_train, y_train)
    r_score = model.score(X_test, y_test)
    return (model, r_score)

# それぞれの特徴量に対してモデルを学習し、テストスコアを算出する
(m1, r1) = evaluate_feature(X1_train, X1_test, y_train, y_test)
(m2, r2) = evaluate_feature(X2_train, X2_test, y_train, y_test)
print("R-squared score with singleton features: %0.5f" % r1)
print("R-squared score with pairwise features: %0.10f" % r2)

R-squared score with singleton features: 0.00924
R-squared score with pairwise features: 0.0113190914


* 交互作用特徴量のモデル学習コストは増大する

    * 例)元の特徴量が$n$個ある場合、ペアワイズ交互作用特徴量は$n^2$個作成される
    
    * 結果として、線形モデルの学習時間とテストスコア算出時間は$O(n)$から$O(n^2)$に増大する

* 交互作用特徴量を利用する際に、計算時間の増大を回避する方法がある

    * 1つは、交互作用特徴量に対して特徴選択を実施する方法
    
    * もう1つは、手作業で複合的な特徴量を注意深く作ることで、特徴量の数を押される方法

* ただし、どちらの戦法にもメリットとデメリットがある

* 特徴選択は問題に適した特徴量を自動的に選択する手法

    * これは交互作用特徴量に限らず多数の特徴量を少数に絞るために使える
    
    * しかし、特徴選択の手法には特徴量の数が大きいままモデルを何度も学習する必要がある

* 手作業で作られた特徴量は、表現力が高くなり、特徴量の数を少なく抑えることができる

    * したがって、モデルの学習時間が削減できる
    
    * しかし、特徴量自体の計算に時間がかかるため、テストスコアを算出するための計算コストが増大する

| 版   | 年/月/日   |
| ---- | ---------- |
| 初版 | 2019/04/14 |